In [ ]:
# | default_exp _components.helpers

# Internal helpers

In [ ]:
# | export


def in_notebook() -> bool:
    try:
        from IPython import get_ipython

        if "IPKernelApp" not in get_ipython().config:
            return False
    except ImportError:
        return False
    except AttributeError:
        return False
    return True

In [ ]:
in_notebook()

True

In [ ]:
# | export

import contextlib
import importlib
import os
import sys
from functools import wraps
from inspect import signature
from pathlib import Path
from typing import *

import docstring_parser
import nbformat
import typer
from fastcore.meta import delegates
from nbconvert import PythonExporter

from fastkafka._components.logger import get_logger

In [ ]:
from tempfile import TemporaryDirectory

from aiokafka import AIOKafkaConsumer, AIOKafkaProducer
from nbdev_mkdocs.docstring import run_examples_from_docstring

from fastkafka._application.app import FastKafka
from fastkafka._components.logger import supress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export

F = TypeVar("F", bound=Callable[..., Any])


def _format_args(xs: List[docstring_parser.DocstringParam]) -> str:
    return "\nArgs:\n - " + "\n - ".join(
        [f"{x.arg_name} ({x.type_name}): {x.description}" for x in xs]
    )


def combine_params(f: F, o: Union[Type, Callable[..., Any]]) -> F:
    """Combines docstring arguments of a function and another object or function

    Args:
        f: destination functions where combined arguments will end up
        o: source function from which arguments are taken from

    Returns:
        Function f with augumented docstring including arguments from both functions/objects
    """
    src_params = docstring_parser.parse_from_object(o).params
    #     logger.info(f"combine_params(): source:{_format_args(src_params)}")
    docs = docstring_parser.parse_from_object(f)
    #     logger.info(f"combine_params(): destination:{_format_args(docs.params)}")
    dst_params_names = [p.arg_name for p in docs.params]

    combined_params = docs.params + [
        x for x in src_params if not x.arg_name in dst_params_names
    ]
    #     logger.info(f"combine_params(): combined:{_format_args(combined_params)}")

    docs.meta = [
        x for x in docs.meta if not isinstance(x, docstring_parser.DocstringParam)
    ] + combined_params  # type: ignore

    f.__doc__ = docstring_parser.compose(
        docs, style=docstring_parser.DocstringStyle.GOOGLE
    )
    return f

In [ ]:
def f2(a: int, b: str):
    """
    Args:
        a: parameter a
        b: parameter bbbb
    """


def f1(b: str, c: int):
    """Function f1
    Args:
        b: parameter b
        c: parameter c

    Raises:
        ValueError: sometimes
    """


combine_params(f1, f2).__doc__

expected = """Function f1
Args:
    b: parameter b
    c: parameter c
    a: parameter a

Raises:
    ValueError: sometimes"""

assert f1.__doc__ == expected

In [ ]:
# | export


def delegates_using_docstring(o: Union[Type, Callable[..., Any]]) -> Callable[[F], F]:
    def _delegates_using_docstring(f: F) -> F:
        def _combine_params(o: Union[Type, Callable[..., Any]]) -> Callable[[F], F]:
            def __combine_params(f: F, o: Union[Type, Callable[..., Any]] = o) -> F:
                return combine_params(f=f, o=o)

            return __combine_params

        @_combine_params(o)
        @delegates(o)  # type: ignore
        @wraps(f)
        def _f(*args: Any, **kwargs: Any) -> Any:
            return f(*args, **kwargs)

        return _f

    return _delegates_using_docstring

In [ ]:
def f2(a: str, d: int) -> None:
    """
    Args:
        a: parameter a
        b: parameter bbbb
    """
    pass


@delegates_using_docstring(f2)
def f1(b: str, c: int, **kwargs):
    """Function f1
    Args:
        b: parameter b
        c: parameter c

    Raises:
        ValueError: sometimes
    """
    pass


expected = """Function f1
Args:
    b: parameter b
    c: parameter c
    a: parameter a

Raises:
    ValueError: sometimes"""
assert f1.__doc__ == expected

In [ ]:
signature(f1).parameters, signature(f2).parameters

(mappingproxy({'b': <Parameter "b: str">, 'c': <Parameter "c: int">}),
 mappingproxy({'a': <Parameter "a: str">, 'd': <Parameter "d: int">}))

In [ ]:
@delegates(f2)
def f3(b: str, c: int, **kwargs):
    """Function f1
    Args:
        b: parameter b
        c: parameter c

    Raises:
        ValueError: sometimes
    """
    pass


signature(f3).parameters

mappingproxy({'b': <Parameter "b: str">, 'c': <Parameter "c: int">})

In [ ]:
@delegates_using_docstring(AIOKafkaConsumer)
def f(a: int, **kwargs) -> str:
    """function a
    Args:
        a: parameter a

    Returns:
        stuff
    """
    print(f"{a=}")

In [ ]:
print(f.__doc__)

function a
Args:
    a: parameter a
    *topics (list(str)): optional list of topics to subscribe to. If not set,
        call :meth:`.subscribe` or :meth:`.assign` before consuming records.
        Passing topics directly is same as calling :meth:`.subscribe` API.
    bootstrap_servers (str, list(str)): a ``host[:port]`` string (or list of
        ``host[:port]`` strings) that the consumer should contact to bootstrap
        initial cluster metadata.
        
        This does not have to be the full node list.
        It just needs to have at least one broker that will respond to a
        Metadata API Request. Default port is 9092. If no servers are
        specified, will default to ``localhost:9092``.
    client_id (str): a name for this client. This string is passed in
        each request to servers and can be used to identify specific
        server-side log entries that correspond to this client. Also
        submitted to :class:`~.consumer.group_coordinator.GroupCoordinator`


In [ ]:
signature(f).parameters

mappingproxy({'a': <Parameter "a: int">,
              'loop': <Parameter "loop=None">,
              'bootstrap_servers': <Parameter "bootstrap_servers='localhost'">,
              'client_id': <Parameter "client_id='aiokafka-0.8.0'">,
              'group_id': <Parameter "group_id=None">,
              'key_deserializer': <Parameter "key_deserializer=None">,
              'value_deserializer': <Parameter "value_deserializer=None">,
              'fetch_max_wait_ms': <Parameter "fetch_max_wait_ms=500">,
              'fetch_max_bytes': <Parameter "fetch_max_bytes=52428800">,
              'fetch_min_bytes': <Parameter "fetch_min_bytes=1">,
              'max_partition_fetch_bytes': <Parameter "max_partition_fetch_bytes=1048576">,
              'request_timeout_ms': <Parameter "request_timeout_ms=40000">,
              'retry_backoff_ms': <Parameter "retry_backoff_ms=100">,
              'auto_offset_reset': <Parameter "auto_offset_reset='latest'">,
              'enable_auto_commit': 

In [ ]:
# | export


def use_parameters_of(
    o: Union[Type, Callable[..., Any]], **kwargs: Dict[str, Any]
) -> Dict[str, Any]:
    """Restrict parameters passwed as keyword arguments to parameters from the signature of ``o``

    Args:
        o: object or callable which signature is used for restricting keyword arguments
        kwargs: keyword arguments

    Returns:
        restricted keyword arguments

    """
    allowed_keys = set(signature(o).parameters.keys())
    return {k: v for k, v in kwargs.items() if k in allowed_keys}

In [ ]:
assert use_parameters_of(AIOKafkaConsumer, api_version=0.1, radnom_param="random") == {
    "api_version": 0.1
}

In [ ]:
# | export


def generate_app_src(out_path: Union[Path, str]) -> None:
    path = Path("099_Test_Service.ipynb")
    if not path.exists():
        path = Path("..") / "099_Test_Service.ipynb"
    if not path.exists():
        raise ValueError(f"Path '{path.resolve()}' does not exists.")

    with open(path, "r") as f:
        notebook = nbformat.reads(f.read(), nbformat.NO_CONVERT)
        exporter = PythonExporter()
        source, _ = exporter.from_notebook_node(notebook)

    with open(out_path, "w") as f:
        f.write(source)

In [ ]:
with TemporaryDirectory() as d:
    generate_app_src((Path(d) / "main.py"))
    !ls -al {d}
    !cat {d}/main.py | grep @kafka_app

total 20
drwx------ 2 davor davor 4096 Mar 15 09:19 .
drwxrwxrwt 1 root  root  4096 Mar 15 09:19 ..
-rw-rw-r-- 1 davor davor 9888 Mar 15 09:19 main.py
    @kafka_app.consumes()  # type: ignore
    @kafka_app.consumes()  # type: ignore
    @kafka_app.produces()  # type: ignore
    @kafka_app.produces()  # type: ignore
    @kafka_app.produces()  # type: ignore
    @kafka_app.produces()  # type: ignore


In [ ]:
# | export


@contextlib.contextmanager
def change_dir(d: str) -> Generator[None, None, None]:
    curdir = os.getcwd()
    os.chdir(d)
    try:
        yield
    finally:
        os.chdir(curdir)

In [ ]:
with TemporaryDirectory() as d:
    original_wd = os.getcwd()
    assert original_wd != d
    with change_dir(d):
        assert os.getcwd() == d
    assert os.getcwd() == original_wd

In [ ]:
# | export


class ImportFromStringError(Exception):
    pass


def _import_from_string(import_str: str) -> Any:
    """Imports library from string

    Note:
        copied from https://github.com/encode/uvicorn/blob/master/uvicorn/importer.py

    Args:
        import_str: input string in form 'main:app'

    """
    sys.path.append(".")

    if not isinstance(import_str, str):
        return import_str

    module_str, _, attrs_str = import_str.partition(":")
    if not module_str or not attrs_str:
        message = (
            'Import string "{import_str}" must be in format "<module>:<attribute>".'
        )
        typer.secho(f"{message}", err=True, fg=typer.colors.RED)
        raise ImportFromStringError(message.format(import_str=import_str))

    try:
        # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
        module = importlib.import_module(module_str)
    except ImportError as exc:
        if exc.name != module_str:
            raise exc from None
        message = 'Could not import module "{module_str}".'
        raise ImportFromStringError(message.format(module_str=module_str))

    instance = module
    try:
        for attr_str in attrs_str.split("."):
            instance = getattr(instance, attr_str)
    except AttributeError:
        message = 'Attribute "{attrs_str}" not found in module "{module_str}".'
        raise ImportFromStringError(
            message.format(attrs_str=attrs_str, module_str=module_str)
        )

    return instance

In [ ]:
with TemporaryDirectory() as d:
    src_path = Path(d) / "main.py"
    generate_app_src(src_path)
    with change_dir(d):
        kafka_app = _import_from_string(f"{src_path.stem}:kafka_app")
        assert isinstance(kafka_app, FastKafka)

In [ ]:
# | export


def filter_using_signature(f: Callable, **kwargs: Dict[str, Any]) -> Dict[str, Any]:
    """todo: write docs"""
    param_names = list(signature(f).parameters.keys())
    return {k: v for k, v in kwargs.items() if k in param_names}

In [ ]:
def f(a: int, *, b: str):
    pass


assert filter_using_signature(f, a=1, c=3) == {"a": 1}